In [337]:
import pandas as pd
import numpy as np

In [338]:
dir_path='J:\\NanTongAPTIV\\newTable\\'
material_fileName='material.xlsx'
mod_resource_fileName='mod_resource.xlsx'
shift_mod_fileName='可变模具.xlsx'
material_path=dir_path+material_fileName
mod_resource_path=dir_path+mod_resource_fileName
shift_mod_path=dir_path+shift_mod_fileName
material=pd.read_excel(material_path)
mod_resource=pd.read_excel(mod_resource_path)
shift_mod_group=pd.read_excel(shift_mod_path,sheet_name='sheet1')
material_group=pd.read_excel(shift_mod_path,sheet_name='sheet2')

In [339]:
#最终要出现的表的样式
shift_mod_efficiency=pd.DataFrame(columns=['resource_code','source_material_code','target_material_code','shift_time'])
err_mod_message=pd.DataFrame(columns=['resource_code','material_group','shift_time'])
err_material_message=pd.DataFrame(columns=['material_group','material_code'])

In [340]:
# 需要建立一个映射关系，模具code←→id，物料code←→id，需要生成类数据库列的Excel表

In [341]:
#数据格式准备
shift_mod_group.rename(columns={'资源编号':'resource_code','物料组别':'material_group','轮换时间(分钟)':'shift_time'},inplace=True)
material_group.rename(columns={'物料组别':'material_group','物料编号':'material_code'},inplace=True)
material_efficiency=material[['id','code']]
mod_resource_efficiency=mod_resource[['id','code']]

In [342]:
#将sheet1的resource_code转化未resource_id，并将错误数据在这里抛出
# 还需增加将错误数据导出到另一张表的操作
for i in range(1,len(shift_mod_group)):
    temp_mod_resource_code_to_id=pd.DataFrame(columns=['id','code'])
    # 拿到resource_id
    temp_mod_resource_code_to_id=mod_resource_efficiency[shift_mod_group['resource_code'][i] == mod_resource_efficiency['code']]
    if (temp_mod_resource_code_to_id.empty):
        err_mod_message=pd.concat([err_mod_message,shift_mod_group.iloc[i,:].to_frame().T])
        shift_mod_group['resource_code'][i]=None
        continue
    shift_mod_group['resource_code'][i]=temp_mod_resource_code_to_id['id'].values[0]
shift_mod_group=shift_mod_group.dropna(subset=['resource_code'])
shift_mod_group.rename(columns={'resource_code':'resource_id'},inplace=True)

In [343]:
# shift_mod_group

In [344]:
#先将sheet2的material_code全转成material_id
# 还需增加将错误数据导出到另一张表的操作
for i in range(1,len(material_group)):
    # 这里后面要加防御，避免出现这里的code不在material里面的情况
    temp_material_group_code_to_id=pd.DataFrame(columns=['id','code'])
    temp_material_group_code_to_id=material_efficiency[material_group['material_code'][i]==material_efficiency['code']]
    if(temp_material_group_code_to_id.empty):
        err_material_message=pd.concat([err_material_message,material_group.iloc[i,:].to_frame().T])
        material_group['material_code'][i]=None
        continue
    material_group['material_code'][i]=temp_material_group_code_to_id['id'].values[0]
material_group=material_group.dropna(subset=['material_code'])
material_group.rename(columns={'material_code':'material_id'},inplace=True)

In [345]:
for i in range(1,len(shift_mod_group)):
    # 这里对数据需要进行一个处理，如果一个组只有一条数据，则导出
    # 从sheet1拿到material_group，拿到sheet2去得到其一个组的所有material_id，准备去做排列
    material_group_value=shift_mod_group['material_group'][i]
    temp_material_group=material_group[material_group_value==material_group['material_group']]
    temp_material_group=temp_material_group.reset_index(drop=True)
    temp_shift_mod=pd.DataFrame(columns=['resource_code','source_material_code','target_material_code','shift_time'],index=range(len(temp_material_group)*(len(temp_material_group)-1)))
    # 在最终要输出的格式的下标需要另外定义，因为存在排列，如果一个组有4个元素，则需要导出A(4,2)=12条数据
    # 两种错误要进行特殊说明1、当sheet1里面的code没有对应id的时候，前面已经把他删除和导出处理了，那么这个物料组别的数据在sheet2理应也要删除
    # 2、material的code不存在删除了也就不进行排列
    temp_index=0
    for j in range(len(temp_material_group)):
        #当下标比这个A(n,2)=n*(n-1)大的时候
        if(temp_index>(len(temp_material_group)*(len(temp_material_group)-1))):
            break
        for s in range(len(temp_material_group)):
            # temp_shift_mod['resource_code'][temp_index]=shift_mod_group['resource_id'][i]
            # 这样报错： index 0 is out of bounds for axis 0 with size 0 是因为我创建了一个空的temp_shift_mod
            # 解决方法1：我在创建的时候根据规则给他创建一个相应大小的dataframe
            temp_shift_mod['resource_code'][temp_index]=shift_mod_group['resource_id'][i]
            # 从sheet1拿到shift_time，对于同一组的它的shift_time应该是不变的
            temp_shift_mod['shift_time'][temp_index]=shift_mod_group['shift_time'][i]
            # 做len(material_group)次循环，每次拿到一个作为source_material_id，剩下的作为target_material_id
            temp_shift_mod['source_material_code'][temp_index]=temp_material_group['material_id'][j]
            if(s==j):
                continue
            else:
                temp_shift_mod['target_material_code'][temp_index]=temp_material_group['material_id'][s]
                temp_index+=1
    #在temp_shift_mod里面已经有A(N,2)条数据了，这时候就和最终的表进行拼接
    shift_mod_efficiency=pd.concat([shift_mod_efficiency,temp_shift_mod],ignore_index=True)
shift_mod_efficiency=shift_mod_efficiency.dropna()
temp_shift_mod=pd.DataFrame(['resource_code','source_material_code','target_material_code','shift_time']).T
temp_shift_mod.columns=shift_mod_efficiency.columns
shift_mod_efficiency=pd.concat([temp_shift_mod,shift_mod_efficiency],axis=0,ignore_index=True)
shift_mod_efficiency.rename(columns={'resource_code':'资源编号','source_material_code':'源物料编号','target_material_code':'目标物料编号','shift_time':'轮换时间(分钟)'},inplace=True)
shift_mod_efficiency.to_excel(dir_path+'可变模具_python.xlsx', sheet_name='sheet1', index=False)
err_material_message.to_excel(dir_path+'可变模具_物料_出错数据_python.xlsx', sheet_name='sheet1', index=False)
err_mod_message.to_excel(dir_path+'可变模具_模具_出错数据_python.xlsx', sheet_name='sheet1', index=False)